In [18]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

In [3]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(
    r'D:\AI nutrition analyzer\Dataset\TEST_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(
    r'D:\AI nutrition analyzer\Dataset\TEST_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 1029 images belonging to 5 classes.
Found 1029 images belonging to 5 classes.


In [4]:
print(x_train.class_indices)#checking the number of classes
print(x_test.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}
{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [5]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 217, 1: 415, 2: 248, 3: 70, 4: 79})

In [6]:
from collections import Counter as c
c(x_test .labels)

Counter({0: 217, 1: 415, 2: 248, 3: 70, 4: 79})

In [7]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
c
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [8]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6

In [9]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [10]:
classifier.fit(
        x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 206 steps, validate for 206 steps
Epoch 1/20
206/206 [==============================] - 14s 67ms/step - loss: 1.1980 - accuracy: 0.5403 - val_loss: 0.9148 - val_accuracy: 0.6686
Epoch 2/20
206/206 [==============================] - 11s 53ms/step - loss: 1.0019 - accuracy: 0.6375 - val_loss: 0.8460 - val_accuracy: 0.7007
Epoch 3/20
206/206 [==============================] - 11s 52ms/step - loss: 0.9196 - accuracy: 0.6667 - val_loss: 0.9663 - val_accuracy: 0.6229
Epoch 4/20
206/206 [==============================] - 10s 51ms/step - loss: 0.8728 - accuracy: 0.6638 - val_loss: 0.7840 - val_accuracy: 0.7133
Epoch 5/20
206/206 [==============================] - 11s 51ms/step - loss: 0.8067 - accuracy: 0.6890 - val_loss: 0.7094 - val_accuracy: 0.7444
Epoch 6/20
206/206 [==============================] - 11s 52ms/step - loss: 0.7200 - accuracy: 0.7347 - val_loss: 0.6471 - val_accuracy: 0.7648
Epoch 7/20
206/206 [======================

In [23]:
classifier.save('saved_model.h5')   # Saving model

ERROR! Session/line number was not unique in database. History logging moved to new session 42


In [15]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("saved_model.h5") #loading the model for testing

ERROR! Session/line number was not unique in database. History logging moved to new session 35


In [16]:
img = image.load_img(r"D:\Dataset\TRAIN_SET\ORANGE\n07749192_97.jpg",
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred=np.argmax(model.predict(x), axis=1)
pred

array([2], dtype=int64)

In [17]:
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
result=str(index[pred[0]])
result

'ORANGE'